<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Frsm/notebooks/rsm_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with RSM (Mansfield Relative Strength) Rating

### Install and Config

#### Install Required Packages

In [8]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/rsm"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/rsm) to /tmp/pip-req-build-35d9xho2
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-35d9xho2
  Running command git checkout -b feature/rsm --track origin/feature/rsm
  Switched to a new branch 'feature/rsm'
  Branch 'feature/rsm' set up to track remote branch 'feature/rsm' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit cf6b56bfe9f9bdd2968933bcbbc05d927e4d3209
  Preparing metadata (setup.py) ... done


#### Setup and Configuration

In [9]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [10]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [11]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Cannot download.")
            return None

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers)
        if file_info:
            download_url = file_info['download_url']
            response = requests.get(download_url)
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
                return None
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            sha = file_info['sha']
            payload = {
                'message': 'Deleting file',
                'sha': sha,
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='rsm',
)

In [12]:
# @title DataFrame Operations
import vistock.tw as t

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    if df.empty:
        return df
    if not is_taiwan_stock_df(df):
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]   # create a new DataFrame
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [13]:
# @title Rank Function

import os
from datetime import datetime

from vistock import rsm
from vistock import tw
from vistock.stock_indices import get_tickers

def rank(code, period='2y', ma="EMA", tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    #remove_tickers = ['HBAN', 'SW', 'BRK.B', 'VLTO', 'ARM', 'SOLV', 'GEV', 'BF.B']
    #remove_tickers += ['CLX']
    #tickers = [t for t in tickers if t not in remove_tickers]

    rank_stock = rsm.ranking(tickers, period=period,
                             ticker_ref=ticker_ref, ma=ma)
    if rank_stock.empty:
        print("Not enough data to generate rankings.")
        return

    rank_stock = add_name_column(rank_stock)

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    filename = f'{code}_stocks_{period}_{ma}_{today}.csv'
    github.upload_df_as_csv(filename, rank_stock)
    rank_stock.to_csv(os.path.join(out_dir, filename), index=False)
    print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock

### RS Rating and Ranking

In [14]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
ma = "EMA" # @param ["EMA","SMA"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

df = rank(code_from_name[source], ma=ma)
data_table.DataTable(df, include_index=False, num_rows_per_page=20)

[**********************88%%****************      ]  463 of 529 completed

$BF.B: possibly delisted; No price data found  (period=2y)


[*********************100%%**********************]  529 of 529 completed
ERROR:yfinance:
7 Failed downloads:
ERROR:yfinance:['SW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
ERROR:yfinance:['ARM', 'VLTO']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']")
ERROR:yfinance:['GEV', 'SOLV']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']")
ERROR:yfinance:['BRK.B']: YFChartError('%ticker%: No data found, symbol may be delisted')
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (period=2y)')




***
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "SPX+DJIA+NDX+SOX_stocks_2y_EMA_20240824.csv" is in the "out" folder.
***



,Ticker,Price,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Rank,1 Month Ago,3 Months Ago,6 Months Ago
9,NVDA,129.37,39.99,39.26,39.39,57.02,1,1,4,2
373,IRM,113.35,27.92,19.80,7.89,7.29,2,6,67,85
337,HWM,97.08,27.66,10.81,30.59,17.99,3,28,6,24
76,COHR,79.38,25.90,20.53,9.07,22.80,4,4,58,15
417,FICO,1745.38,24.98,23.10,20.24,20.04,5,3,14,21
...,...,...,...,...,...,...,...,...,...,...
194,SW,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
210,BRK.B,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
269,VLTO,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
326,SOLV,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>


### RS Rating and Ranking for Taiwan Stocks

In [15]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
ma = "EMA" # @param ["EMA","SMA"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

df_tw = rank(code_from_name[source], ma=ma,
             tickers_getter=tw.get_tickers, ticker_ref='^TWII')
data_table.DataTable(df_tw, include_index=False, num_rows_per_page=20)

[*********************100%%**********************]  2075 of 2075 completed
ERROR:yfinance:
69 Failed downloads:
ERROR:yfinance:['00941.TW', '8487.TW', '00946.TW', '8162.TW', '00945B.TW', '1563.TW', '6949.TW', '6914.TW', '4771.TW', '6771.TW', '6423.TW', '00944.TW', '00939.TW', '6794.TW', '6928.TW', '00940.TW', '00943.TW', '4949.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']")
ERROR:yfinance:['6890.TW', '00949.TW', '00951.TW', '6957.TW', '3150.TW', '00947.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
ERROR:yfinance:['020029.TW', '020020.TW', '020031.TW', '020000.TW', '020036.TW', '020019.TW', '02001R.TW', '02001L.TW', '020030.TW', '020015.TW', '020012.TW', '020028.TW', '02001S.TW', '020018.TW', '020038.TW', '020034.TW', '020011.TW', '020037.TW', '020039.TW', '020016.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one



***
Your "TWSE+TPEX_stocks_2y_EMA_20240824.csv" is in the "out" folder.
***



,Ticker,Name,Price,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Rank,1 Month Ago,3 Months Ago,6 Months Ago
872,6144,6144,105.0,193.50,176.96,9.10,-14.48,1,2,262,1623
16,1799,1799,229.0,143.78,143.17,56.49,54.10,2,3,21,35
419,3230,3230,41.0,135.12,55.24,20.66,-5.65,3,39,129,911
452,6640,6640,968.0,120.26,105.16,53.11,82.31,4,8,23,12
1226,8374,羅昇,136.0,108.18,118.43,-13.27,-15.51,5,6,1183,1686
...,...,...,...,...,...,...,...,...,...,...,...
1927,00947,台新臺灣IC設計,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1952,6805,富世達,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1953,020016,統一MSCI美科技N,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1965,00943,兆豐電子高息等權,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>


### Remove files in GitHub Repository

In [16]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    if not dates:
        file_selector_widget.options = []
        return
    selected_date = date_selector_widget.value
    if selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    date_selector_widget.options = dates
    date_selector_widget.value = selected_date
    if not dates:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0] if dates else None,
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget, file_selector_widget, delete_button)

Dropdown(description='Date:', options=('20240824', '20240823'), value='20240824')

SelectMultiple(description='Files', layout=Layout(width='430px'), options=('SPX+DJIA+NDX+SOX_stocks_2y_EMA_202…

Button(description='Delete Selected Files', style=ButtonStyle())